# Shelter Animal Outcomes 10

## Gradient Tree Boosting

In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation
import pandas as pd

In [2]:
df_train = pd.read_csv('../Shelter_train.csv')
df_test = pd.read_csv('../Shelter_test.csv')

In [3]:
x = df_train.ix[:, :-1]
y = df_train.ix[:, -1]
df_test = df_test.drop('ID', 1)

In [4]:
clf = GradientBoostingClassifier()
cross_validation.cross_val_score(clf, x, y, scoring="log_loss")

array([-0.85764033, -0.85404969, -0.84077597])

In [5]:
%timeit clf.fit(x, y)

1 loop, best of 3: 6.67 s per loop


In [6]:
clf = clf.fit(x, y)
predictions = clf.predict_proba(df_test)
output = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output.index.names = ['ID']
output.index += 1
output.head()

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.058208,0.004426,0.058155,0.242816,0.636396
2,0.502340,0.001657,0.031773,0.301103,0.163128
3,0.474455,0.002123,0.015979,0.110612,0.396830
4,0.093138,0.004179,0.033231,0.152507,0.716946
5,0.437836,0.001805,0.035853,0.339781,0.184724


In [7]:
output.to_csv('../submission-GradientBoostingClassifier.1.0.csv', index_label = 'ID')